In [ ]:
import torch
import torch.nn.functional as F

# 默认查找1的序列，默认值为0，1，2
raw_input = torch.tensor([
    [0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
    [1, 1, 1, 1, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 1, 1, 0],
    [0, 1, 0, 1, 0, 0, 1, 0, 0, 1],
    [0, 0, 1, 0, 1, 0, 1, 1, 1, 1],
    [0, 1, 1, 0, 1, 0, 0, 1, 0, 1],
    [0, 1, 0, 0, 0, 1, 0, 0, 0, 1],
    [0, 0, 0, 1, 0, 1, 1, 1, 0, 0],
    [0, 1, 0, 1, 1, 0, 1, 1, 1, 0],
    [0, 0, 0, 1, 1, 1, 1, 0, 0, 0]
    ], dtype=torch.float32)
# 边缘增加一圈2
size = raw_input.shape[0]
mat = -1 * torch.ones((size+2, size+2))
mat[1:-1, 1:-1] = raw_input
mat = mat.unsqueeze(0).unsqueeze(0)
mat

In [ ]:
row_kernel = torch.tensor([
    [[[1, 1, 0, 1, 1, 1]]],
    [[[0, 1, 1, 1, 1, 0]]],
    ], dtype=torch.float32)
print(row_kernel.shape)
F.conv2d(mat, row_kernel)

In [ ]:
%%timeit

import numpy as np
raw_input = np.array([
    [0, 0, 0, 1, 1, 0, 0, 0, 0, 1],
    [1, 1, 1, 1, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 1, 1, 0],
    [0, 1, 0, 1, 0, 0, 1, 0, 0, 1],
    [0, 0, 1, 0, 1, 0, 1, 1, 1, 1],
    [0, 1, 1, 0, 1, 0, 0, 1, 0, 1],
    [0, 1, 0, 0, 0, 1, 0, 0, 0, 1],
    [0, 0, 0, 1, 0, 1, 1, 1, 0, 0],
    [0, 1, 0, 1, 1, 0, 1, 1, 1, 0],
    [0, 0, 0, 1, 1, 1, 1, 0, 0, 0]
    ], dtype=np.int8)
kernel = np.array([1, 1, 0, 1, 1], dtype=np.int8)
for i in range(raw_input.shape[0]):
    for j in range(5, raw_input.shape[0]+1):
        res = (raw_input[i, j-5:j] == kernel).all()


In [115]:
def hori_match_arr(matrix, arr, all=False):
    """遍历搜索二维数组, 查找数组arr是否存在
    如果存在,返回序列的坐标
    如果all=True, 则需要查找所有满足要求的序列
    """
    n, k = len(matrix), len(arr)
    if not all: # 无需返回所有存在的序列索引
        for i in range(n):
            for j in range(n-k+1):
                if matrix[i][j:j+k] == arr:
                    return (i, j)
    else: # 需要返回所有
        indexes = []
        for i in range(n):
            for j in range(n-k+1):
                if matrix[i][j:j+k] == arr:
                    indexes.append((i, j))
        return indexes


def vert_match_arr(matrix, arr, all=False):
    """遍历搜索二维数组, 查找数组arr是否存在
    如果存在,返回序列的坐标
    如果all=True, 则需要查找所有满足要求的序列
    """
    n, k = len(matrix), len(arr)
    if not all: # 无需返回所有存在的序列索引
        for j in range(n):
            for i in range(n-k+1):
                sub = [matrix[i+m][j] for m in range(k)]
                if sub == arr:
                    return (i, j)
    else: # 需要返回所有
        indexes = []
        for j in range(n):
            for i in range(n-k+1):
                sub = [matrix[i+m][j] for m in range(k)]
                if sub == arr:
                    indexes.append((i, j))
        return indexes


def diag_match_arr(matrix, arr, all=False):
    """遍历搜索二维数组, 查找数组arr是否存在
    如果存在,返回序列的坐标
    如果all=True, 则需要查找所有满足要求的序列
    """
    n, k = len(matrix), len(arr)
    stop_k = n - k + 1
    if not all: # 无需返回所有存在的序列索引
        for x in range(n-k, 0, -1): # 右半轴
            for i, j in enumerate(range(x, stop_k)):
                sub = [matrix[i+m][j+m] for m in range(k)]
                if sub == arr:
                    return (i, j)
        for x in range(stop_k): # 左半轴
            for i, j in zip(range(x, stop_k), range(min(n-x, stop_k))):
                sub = [matrix[i+m][j+m] for m in range(k)]
                if sub == arr:
                    return (i, j)
    else: # 需要返回所有
        indexes = []
        for x in range(n-k, 0, -1): # 右半轴
            for i, j in enumerate(range(x, stop_k)):
                sub = [matrix[i+m][j+m] for m in range(k)]
                if sub == arr:
                    indexes.append((i, j))
        for x in range(stop_k): # 左半轴
            for i, j in zip(range(x, stop_k), range(min(n-x, stop_k))):
                sub = [matrix[i+m][j+m] for m in range(k)]
                if sub == arr:
                    indexes.append((i, j))
        return indexes


def rdiag_match_arr(matrix, arr, all=False):
    """遍历搜索二维数组, 查找数组arr是否存在
    如果存在,返回序列的坐标
    如果all=True, 则需要查找所有满足要求的序列
    """
    n, k = len(matrix), len(arr)
    stop_k = n - k + 1
    if not all: # 无需返回所有存在的序列索引
        for x in range(n-k-1, n): # 左半轴
            for i, j in zip(range(x), range(x, k-2, -1)):
                sub = [matrix[i+m][j-m] for m in range(k)]
                if sub == arr:
                    return (i, j)
        for x in range(stop_k): # 右半轴
            for i, j in zip(range(x, stop_k), range(n-1, k-2, -1)):
                sub = [matrix[i+m][j-m] for m in range(k)]
                if sub == arr:
                    return (i, j)
    else: # 需要返回所有
        indexes = []
        for x in range(n-k-1, n): # 左半轴
            for i, j in zip(range(x), range(x, k-2, -1)):
                sub = [matrix[i+m][j-m] for m in range(k)]
                if sub == arr:
                    indexes.append((i, j))
        for x in range(stop_k): # 右半轴
            for i, j in zip(range(x, stop_k), range(n-1, k-2, -1)):
                sub = [matrix[i+m][j-m] for m in range(k)]
                if sub == arr:
                    indexes.append((i, j))
        return indexes



In [119]:

from collections import defaultdict as ddict

class Direction(object):
    """方向定义"""
    HORI = 0
    VERT = 1
    DIAG = 2
    RDIAG = 3


class Matcher(object):
    """在矩阵中搜索给定的数组"""
    def __init__(self, matrix, arr, match_all=False) -> None:
        self.matrix = matrix
        self.arr = arr
        self.rt_all = match_all
        self.n, self.k = len(matrix), len(arr)
        self.indexes = ddict(list)

    def _hori_match_arr(self):
        """水平方向"""
        for i in range(self.n):
            for j in range(self.n-self.k+1):
                if self.matrix[i][j:j+self.k] != self.arr:
                    continue
                self.indexes[Direction.HORI].append((i, j))
                if not self.rt_all:
                    return

    def _vert_match_arr(self):
        """竖直方向"""
        for j in range(self.n):
            for i in range(self.n-self.k+1):
                sub = [self.matrix[i+m][j] for m in range(self.k)]
                if sub != self.arr:
                    continue
                self.indexes[Direction.VERT].append((i,j))
                if not self.rt_all:
                    return

    def _diag_match_arr(self):
        """对角线"""
        stop_k = self.n - self.k + 1
        for x in range(self.n-self.k, 0, -1): # 右半轴
            for i, j in enumerate(range(x, stop_k)):
                sub = [self.matrix[i+m][j+m] for m in range(self.k)]
                if sub != self.arr:
                    continue
                self.indexes[Direction.DIAG].append((i, j))
                if not self.rt_all:
                    return
        for x in range(stop_k): # 左半轴
            for i, j in zip(range(x, stop_k), range(min(self.n-x, stop_k))):
                sub = [self.matrix[i+m][j+m] for m in range(self.k)]
                if sub != self.arr:
                    continue
                self.indexes[Direction.DIAG].append((i, j))
                if not self.rt_all:
                    return

    def _rdiag_match_arr(self):
        """逆对角线"""
        stop_k = self.n - self.k + 1
        for x in range(self.n-self.k-1, self.n): # 左半轴
            for i, j in zip(range(x), range(x, self.k-2, -1)):
                sub = [self.matrix[i+m][j-m] for m in range(self.k)]
                if sub != self.arr:
                    continue
                self.indexes[Direction.RDIAG].append((i, j))
                if not self.rt_all:
                    return
        for x in range(stop_k): # 右半轴
            for i, j in zip(range(x, stop_k), range(self.n-1, self.k-2, -1)):
                sub = [self.matrix[i+m][j-m] for m in range(self.k)]
                if sub != self.arr:
                    continue
                self.indexes[Direction.RDIAG].append((i, j))
                if not self.rt_all:
                    return

    def match_arr(self):
        """遍历搜索二维数组, 查找数组arr是否存在
        如果存在,返回序列的坐标
        如果rt_all=True, 则需要查找所有满足要求的序列
        """
        self._hori_match_arr()
        if not self.rt_all and len(self.indexes[Direction.HORI]):
            return self.indexes[Direction.HORI][0]
        self._vert_match_arr()
        if not self.rt_all and len(self.indexes[Direction.VERT]):
            return self.indexes[Direction.VERT][0]
        self._diag_match_arr()
        if not self.rt_all and len(self.indexes[Direction.DIAG]):
            return self.indexes[Direction.DIAG][0]
        self._rdiag_match_arr()
        if not self.rt_all and len(self.indexes[Direction.RDIAG]):
            return self.indexes[Direction.RDIAG][0]
        if not self.rt_all or not len(self.indexes):
            return False
        return self.indexes


In [125]:
# %%timeit

raw = [
    [0, 0, 0, 1, 1, 0, 0, 0, 0, 1],
    [1, 1, 1, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 1, 1, 0],
    [0, 1, 0, 1, 0, 0, 1, 1, 0, 1],
    [0, 0, 1, 1, 1, 0, 1, 1, 1, 1],
    [1, 1, 1, 0, 1, 1, 0, 1, 0, 1],
    [1, 0, 0, 0, 0, 1, 0, 0, 0, 1],
    [0, 0, 1, 1, 1, 1, 1, 1, 0, 0],
    [0, 1, 1, 1, 1, 0, 1, 1, 1, 0],
    [0, 0, 0, 1, 1, 1, 1, 0, 0, 0]
]
kernel = [1,0,1,1,1]

matcher = Matcher(raw, kernel, True)
index = matcher.match_arr()
index

defaultdict(list,
            {0: [(4, 4), (8, 4)],
             1: [(0, 3), (5, 4)],
             2: [(1, 5), (1, 1), (5, 1), (5, 0)],
             3: [(1, 5), (4, 7), (5, 7), (5, 9)]})

In [126]:
# %%timeit

raw = [
    [0, 0, 0, 1, 1, 0, 0, 0, 0, 1],
    [1, 1, 1, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 1, 1, 0],
    [0, 1, 0, 1, 0, 0, 1, 1, 0, 1],
    [0, 0, 1, 1, 1, 0, 1, 1, 1, 1],
    [1, 1, 1, 0, 1, 1, 0, 1, 0, 1],
    [1, 0, 0, 0, 0, 1, 0, 0, 0, 1],
    [0, 0, 1, 1, 1, 1, 1, 1, 0, 0],
    [0, 1, 1, 1, 1, 0, 1, 1, 1, 0],
    [0, 0, 0, 1, 1, 1, 1, 0, 0, 0]
]
kernel = [1,0,1,1,1]
rt = True
h = hori_match_arr(raw, kernel, True)
v = vert_match_arr(raw, kernel, True)
d = diag_match_arr(raw, kernel, True)
r = rdiag_match_arr(raw, kernel, True)
print('hori', h)
print('vert', v)
print('diag', d)
print('rdiag', r)


hori [(4, 4), (8, 4)]
vert [(0, 3), (5, 4)]
diag [(1, 5), (1, 1), (5, 1), (5, 0)]
rdiag [(1, 5), (4, 7), (5, 7), (5, 9)]


In [16]:
%%timeit

raw = [
    [0, 0, 0, 1, 1, 0, 0, 0, 0, 1],
    [1, 1, 1, 1, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 1, 1, 0],
    [0, 1, 0, 1, 0, 0, 1, 0, 0, 1],
    [0, 0, 1, 0, 1, 0, 1, 1, 1, 1],
    [0, 1, 1, 0, 1, 0, 0, 1, 0, 1],
    [0, 1, 0, 0, 0, 1, 0, 0, 0, 1],
    [0, 0, 0, 1, 0, 1, 1, 1, 0, 0],
    [0, 1, 0, 1, 1, 0, 1, 1, 1, 0],
    [0, 0, 0, 1, 1, 1, 1, 0, 0, 0]
]
kernel = [1,0,1,1,1]
search2(raw, kernel)


26.4 µs ± 386 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [7]:
from PIL import Image

path = "gobang/assets/img/black.png"
img = Image.open(path)


'LA'